# ПРЕОБРАЗОВАНИЕ ТЕКСТА В ДАТАФРЕЙМ УНИКАЛЬНЫХ СЛОВ И РАЗДЕЛЕНИЕ НА ЧАСТИ РЕЧИ

In [ ]:
# преобразует комментарии в датафрейм уникальных слов с количеством повторений в тексте
class text_to_dict:
    import re
    
    # приведем к нижнему регистру, избавимся от символов, знаков препинания, 
    def preprocess_text(text):
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
        text = re.sub(' +', ' ', text)
        text = re.sub(r'[^\w\s]+|[\d]+', ' ',text)
        return text.strip()

    # !pip install pymorphy2
    import pymorphy2

    # уберем междометия, частицы, союзы, предлоги, местоимения
    def pos(word, morth=pymorphy2.MorphAnalyzer()):
        return morth.parse(word)[0].tag.POS

    # приведем слова в начальную форму
    def normal_form(word, morth=pymorphy2.MorphAnalyzer()):
        return morth.parse(word)[0].normal_form

    # основная функция - вызывает остальные, составляет словарь уникальных слов, счиатет частоту и возвращает DataFrame
    def init(text):
        data = [preprocess_text(t) for t in text]
        functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO'} 
        sentence_out=[]
        data_out=[]
        i=0
        for sentence in data:
            for word in sentence.split():
                if pos(word) not in functors_pos:
                    data_out.append(normal_form(word))

        # составим словарь уникальных слов в тексте
        words=sorted(set(data_out))
        words_dict={}
        for word in words:
            words_dict.setdefault(word, 0)

        # посчитаем частоту
        for word in data_out:
            words_dict[word]+=1

        return pd.DataFrame(sorted(words_dict.items(), key=lambda x: (x[1],x[0]), reverse=True), columns=['слово', 'количество'])